In [18]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer, DataCollatorForTokenClassification, pipeline
from datasets import ClassLabel, Sequence, load_dataset
import evaluate
import numpy as np
import pandas as pd
from spacy import displacy

Load pre-trained SciBERT model and tokenizer

In [19]:
tokenizer = AutoTokenizer.from_pretrained("allenai/scibert_scivocab_uncased")
model = AutoModelForTokenClassification.from_pretrained("allenai/scibert_scivocab_uncased", num_labels=3)

Some weights of the model checkpoint at allenai/scibert_scivocab_uncased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initi

Load dataset and split dataset into training and validation sets

In [20]:
datasets = load_dataset("ade_corpus_v2", "Ade_corpus_v2_drug_ade_relation")

Found cached dataset ade_corpus_v2 (C:/Users/albbl/.cache/huggingface/datasets/ade_corpus_v2/Ade_corpus_v2_drug_ade_relation/1.0.0/940d61334dbfac6b01ac5d00286a2122608b8dc79706ee7e9206a1edb172c559)
100%|██████████| 1/1 [00:00<00:00, 1000.31it/s]


In [21]:
datasets    

DatasetDict({
    train: Dataset({
        features: ['text', 'drug', 'effect', 'indexes'],
        num_rows: 6821
    })
})

In [22]:
datasets["train"][0]

{'text': 'Intravenous azithromycin-induced ototoxicity.',
 'drug': 'azithromycin',
 'effect': 'ototoxicity',
 'indexes': {'drug': {'start_char': [12], 'end_char': [24]},
  'effect': {'start_char': [33], 'end_char': [44]}}}

In [23]:
consolidated_dataset = {}

for row in datasets["train"]:
    if row["text"] in consolidated_dataset:
        consolidated_dataset[row["text"]]["drug_indices_start"].update(row["indexes"]["drug"]["start_char"])
        consolidated_dataset[row["text"]]["drug_indices_end"].update(row["indexes"]["drug"]["end_char"])
        
    else:
        consolidated_dataset[row["text"]] = {
            "text": row["text"],
            "drug": [row["drug"]],
            # use sets because the indices can repeat for various reasons
            "drug_indices_start": set(row["indexes"]["drug"]["start_char"]),
            "drug_indices_end": set(row["indexes"]["drug"]["end_char"])
        }

df = pd.DataFrame(list(consolidated_dataset.values()))
# for this trial use small subset
df = df[:50]
df.head()

,text,drug,drug_indices_start,drug_indices_end
0,Intravenous azithromycin-induced ototoxicity.,[azithromycin],{12},{24}
1,"Immobilization, while Paget's bone disease was...",[dihydrotachysterol],{91},{109}
2,Unaccountable severe hypercalcemia in a patien...,[dihydrotachysterol],{84},{102}
3,METHODS: We report two cases of pseudoporphyri...,[naproxen],"{58, 71}","{80, 66}"
4,"Naproxen, the most common offender, has been a...",[Naproxen],{0},{8}


In [24]:
df["drug_indices_start"] = df["drug_indices_start"].apply(list).apply(sorted)
df["drug_indices_end"] = df["drug_indices_end"].apply(list).apply(sorted)
df.head()

,text,drug,drug_indices_start,drug_indices_end
0,Intravenous azithromycin-induced ototoxicity.,[azithromycin],[12],[24]
1,"Immobilization, while Paget's bone disease was...",[dihydrotachysterol],[91],[109]
2,Unaccountable severe hypercalcemia in a patien...,[dihydrotachysterol],[84],[102]
3,METHODS: We report two cases of pseudoporphyri...,[naproxen],"[58, 71]","[66, 80]"
4,"Naproxen, the most common offender, has been a...",[Naproxen],[0],[8]


In [25]:
# save to JSON to then import into Dataset object
df.to_json("./data/dataset.jsonl", orient="records", lines=True)

cons_dataset = load_dataset("json", data_files="./data/dataset.jsonl")
cons_dataset = cons_dataset["train"].train_test_split()
cons_dataset

Found cached dataset json (C:/Users/albbl/.cache/huggingface/datasets/json/default-a5cef1f4c130dd6b/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)
100%|██████████| 1/1 [00:00<00:00, 478.75it/s]


DatasetDict({
    train: Dataset({
        features: ['text', 'drug', 'drug_indices_start', 'drug_indices_end'],
        num_rows: 375
    })
    test: Dataset({
        features: ['text', 'drug', 'drug_indices_start', 'drug_indices_end'],
        num_rows: 125
    })
})

Token Labeling

O - outside any entity we care about

B-DRUG - the beginning of a DRUG entity

I-DRUG - inside a DRUG entity

In [26]:
label_list = ['O', 'B-DRUG', 'I-DRUG']

custom_seq = Sequence(feature=ClassLabel(num_classes=3, 
                                         names=label_list,
                                         names_file=None, id=None), length=-1, id=None)

cons_dataset["train"].features["ner_tags"] = custom_seq
cons_dataset["test"].features["ner_tags"] = custom_seq

In [27]:
def generate_row_labels(row, verbose=False):
    """ Given a row from the consolidated `Ade_corpus_v2_drug_ade_relation` dataset, 
    generates BIO tags for drug and effect entities. 
    
    """

    text = row["text"]

    labels = []
    label = "O"
    prefix = ""
    
    # while iterating through tokens, increment to traverse all drug and effect spans
    drug_index = 0
    
    tokens = tokenizer(text, return_offsets_mapping=True)

    for n in range(len(tokens["input_ids"])):
        offset_start, offset_end = tokens["offset_mapping"][n]

        # should only happen for [CLS] and [SEP]
        if offset_end - offset_start == 0:
            labels.append(-100)
            continue
        
        if drug_index < len(row["drug_indices_start"]) and offset_start == row["drug_indices_start"][drug_index]:
            label = "DRUG"
            prefix = "B-"
        
        labels.append(label_list.index(f"{prefix}{label}"))
            
        if drug_index < len(row["drug_indices_end"]) and offset_end == row["drug_indices_end"][drug_index]:
            label = "O"
            prefix = ""
            drug_index += 1

        # need to transition "inside" if we just entered an entity
        if prefix == "B-":
            prefix = "I-"
    
    if verbose:
        print(f"{row}\n")
        orig = tokenizer.convert_ids_to_tokens(tokens["input_ids"])
        for n in range(len(labels)):
            print(orig[n], labels[n])
    tokens["labels"] = labels
    
    return tokens

In [28]:
generate_row_labels(cons_dataset["train"][2], verbose=True)

{'text': 'Of the four patients who responded to HU with an increase in total Hb, all reported symptomatic improvement and three have not required further transfusions.', 'drug': ['HU'], 'drug_indices_start': [38], 'drug_indices_end': [40]}

[CLS] -100
of 0
the 0
four 0
patients 0
who 0
responded 0
to 0
hu 1
with 0
an 0
increase 0
in 0
total 0
hb 0
, 0
all 0
reported 0
symptomatic 0
improvement 0
and 0
three 0
have 0
not 0
required 0
further 0
transfusion 0
##s 0
. 0
[SEP] -100


{'input_ids': [102, 131, 111, 1379, 568, 975, 12879, 147, 4000, 190, 130, 1242, 121, 1114, 7009, 422, 355, 1214, 12104, 3523, 137, 874, 360, 302, 1761, 911, 12601, 30113, 205, 103], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'offset_mapping': [(0, 0), (0, 2), (3, 6), (7, 11), (12, 20), (21, 24), (25, 34), (35, 37), (38, 40), (41, 45), (46, 48), (49, 57), (58, 60), (61, 66), (67, 69), (69, 70), (71, 74), (75, 83), (84, 95), (96, 107), (108, 111), (112, 117), (118, 122), (123, 126), (127, 135), (136, 143), (144, 155), (155, 156), (156, 157), (0, 0)], 'labels': [-100, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100]}

In [29]:
labeled_dataset = cons_dataset.map(generate_row_labels)

Fine-tuning

In [30]:
task = "ner" # Should be one of "ner", "pos" or "chunk"
model_checkpoint = "allenai/scibert_scivocab_uncased"
batch_size = 16

In [31]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.05,
    logging_steps=1
)

In [32]:
data_collator = DataCollatorForTokenClassification(tokenizer)

In [33]:
metric = evaluate.load("seqeval")

In [34]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [35]:
trainer = Trainer(
    model,
    args,
    train_dataset=labeled_dataset["train"],
    eval_dataset=labeled_dataset["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics, 

)

In [36]:
trainer.train()

c:\Users\albbl\Documents\Studium\10_Semester\Projekt\nlp_project\.env\Lib\site-packages\transformers\optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  1%|          | 1/120 [00:09<19:34,  9.87s/it]

{'loss': 1.1536, 'learning_rate': 9.916666666666668e-06, 'epoch': 0.04}


  2%|▏         | 2/120 [00:21<20:56, 10.65s/it]

{'loss': 0.9336, 'learning_rate': 9.833333333333333e-06, 'epoch': 0.08}


  2%|▎         | 3/120 [00:28<17:54,  9.18s/it]

...

100%|██████████| 120/120 [16:45<00:00,  7.14s/it]

{'loss': 0.057, 'learning_rate': 0.0, 'epoch': 5.0}


                                                 
100%|██████████| 120/120 [17:03<00:00,  8.53s/it]

{'eval_loss': 0.05464215949177742, 'eval_precision': 0.8280254777070064, 'eval_recall': 0.9090909090909091, 'eval_f1': 0.8666666666666667, 'eval_accuracy': 0.9820559610705596, 'eval_runtime': 17.9349, 'eval_samples_per_second': 6.97, 'eval_steps_per_second': 0.446, 'epoch': 5.0}
{'train_runtime': 1023.8974, 'train_samples_per_second': 1.831, 'train_steps_per_second': 0.117, 'train_loss': 0.1294136055589964, 'epoch': 5.0}


TrainOutput(global_step=120, training_loss=0.1294136055589964, metrics={'train_runtime': 1023.8974, 'train_samples_per_second': 1.831, 'train_steps_per_second': 0.117, 'train_loss': 0.1294136055589964, 'epoch': 5.0})

In [46]:
sequence = "I was treated with furosemid."
ner_pipe = pipeline(task="ner", model=model, tokenizer=tokenizer)
for entity in ner_pipe(sequence):
    print(entity)

{'entity': 'LABEL_0', 'score': 0.9763866, 'index': 1, 'word': 'i', 'start': 0, 'end': 1}
{'entity': 'LABEL_0', 'score': 0.997759, 'index': 2, 'word': 'was', 'start': 2, 'end': 5}
{'entity': 'LABEL_0', 'score': 0.9955455, 'index': 3, 'word': 'treated', 'start': 6, 'end': 13}
{'entity': 'LABEL_0', 'score': 0.9892511, 'index': 4, 'word': 'with', 'start': 14, 'end': 18}
{'entity': 'LABEL_1', 'score': 0.84162325, 'index': 5, 'word': 'fur', 'start': 19, 'end': 22}
{'entity': 'LABEL_2', 'score': 0.94467264, 'index': 6, 'word': '##ose', 'start': 22, 'end': 25}
{'entity': 'LABEL_2', 'score': 0.98947084, 'index': 7, 'word': '##mid', 'start': 25, 'end': 28}
{'entity': 'LABEL_0', 'score': 0.9896006, 'index': 8, 'word': '.', 'start': 28, 'end': 29}


In [37]:
predictions, labels, _ = trainer.predict(labeled_dataset["test"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

100%|██████████| 8/8 [00:15<00:00,  1.91s/it]


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [38]:
effect_ner_model = pipeline(task="ner", model=model, tokenizer=tokenizer, device=-1)

In [39]:
def visualize_entities(sentence):
    tokens = effect_ner_model(sentence)
    entities = []
    
    for token in tokens:
        label = int(token["entity"][-1])
        if label != 0:
            token["label"] = label_list[label]
            entities.append(token)
    
    params = [{"text": sentence,
               "ents": entities,
               "title": None}]
    
    html = displacy.render(params, style="ent", manual=True, options={
        "colors": {
                   "B-DRUG": "#f08080",
                   "I-DRUG": "#f08080",
               },
    })
    

In [40]:
#examples = [
#    "Abortion, miscarriage or uterine hemorrhage associated with misoprostol (Cytotec), a labor-inducing drug.",
#    "Addiction to many sedatives and analgesics, such as diazepam, morphine, etc.",
#    "Birth defects associated with thalidomide",
#    "Bleeding of the intestine associated with aspirin therapy",
#    "Cardiovascular disease associated with COX-2 inhibitors (i.e. Vioxx)",
#    "Deafness and kidney failure associated with gentamicin (an antibiotic)"
#]

# for example in examples:
#     visualize_entities(example)
#     print(f"{'*' * 50}\n")

with open("./data/Albers.txt", "r", encoding="utf8") as file:
    examples = file.readlines()
for example in examples:
    if example != "":
        visualize_entities(example)
        print(f"{'*' * 50}\n")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


**************************************************



**************************************************



**************************************************



**************************************************



...

**************************************************



**************************************************



**************************************************



**************************************************



**************************************************

